In [1]:
import wget, os
from bs4 import BeautifulSoup
import requests

In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

# Function to get the links from a specific date
def get_links_for_date(date_str):
    url = f'https://www.viva.co.id/indeks/{nama_indeks}/all/{date_str}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all articles inside the 'article-list-container'
    article_container = soup.find('div', class_='article-list-container')
    if not article_container:
        return []

    articles = article_container.find_all('a', class_='article-list-title')

    # Collect the links
    links = [article['href'] for article in articles]
    return links

# Looping from 30th November 2024 to 1st November 2024
start_date = datetime(2024, 11, 30)
end_date = datetime(2024, 1, 1)

all_links = set()  # Use a set to ensure uniqueness
date = start_date

while date >= end_date or len(all_links) < 5000:
    date_str = date.strftime('%Y/%m/%d')  # Format date as 'YYYY/MM/DD'
    print(f"Fetching links for {date_str}...")

    links = get_links_for_date(date_str)
    
    # Append '?page=all' to each link
    modified_links = {link + '?page=all' for link in links}  # Use a set to avoid duplicates
    
    # Add to the main set
    all_links.update(modified_links)

    # Stop if 5000 unique links are reached
    if len(all_links) >= 5000:
        break

    # Move to the previous day
    date -= timedelta(days=1)

# Save the links to a text file
with open('article_links_militer.txt', 'w') as file:
    for link in list(all_links)[:5000]:  # Convert set to list for indexing
        file.write(link + '\n')

print(f"Total unique links saved: {len(all_links)}")

Fetching links for 2024/11/30...
Fetching links for 2024/11/29...
Fetching links for 2024/11/28...
Fetching links for 2024/11/27...
Fetching links for 2024/11/26...
Fetching links for 2024/11/25...
Fetching links for 2024/11/24...
Fetching links for 2024/11/23...
Fetching links for 2024/11/22...
Fetching links for 2024/11/21...
Fetching links for 2024/11/20...
Fetching links for 2024/11/19...
Fetching links for 2024/11/18...
Fetching links for 2024/11/17...
Fetching links for 2024/11/16...
Fetching links for 2024/11/15...
Fetching links for 2024/11/14...
Fetching links for 2024/11/13...
Fetching links for 2024/11/12...
Fetching links for 2024/11/11...
Fetching links for 2024/11/10...
Fetching links for 2024/11/09...
Fetching links for 2024/11/08...
Fetching links for 2024/11/07...
Fetching links for 2024/11/06...
Fetching links for 2024/11/05...
Fetching links for 2024/11/04...
Fetching links for 2024/11/03...
Fetching links for 2024/11/02...
Fetching links for 2024/11/01...
Fetching l

In [18]:
import os
import requests

# Read the article links from the file
with open('article_links_militer.txt', 'r') as file:
    links = file.readlines()

# Create a directory to store the downloaded HTML files
download_dir = 'html_files/militer'
os.makedirs(download_dir, exist_ok=True)

# Loop through the links and download each HTML file
for i, link in enumerate(links):
    link = link.strip()  # Remove any leading/trailing whitespace
    if link:  # Ensure the link is not empty
        try:
            # Get the HTML content of the page
            response = requests.get(link)
            response.raise_for_status()  # Raise an error for bad responses

            # Save the HTML content to a file named 1.html, 2.html, ..., n.html
            file_name = f"{i + 1}.html"
            file_path = os.path.join(download_dir, file_name)
            with open(file_path, 'w', encoding='utf-8') as html_file:
                html_file.write(response.text)

            print(f"Downloaded: {file_name}")

        except requests.exceptions.RequestException as e:
            print(f"Failed to download {link}: {e}")

Downloaded: 1.html
Downloaded: 2.html
Downloaded: 3.html
Downloaded: 4.html
Downloaded: 5.html
Downloaded: 6.html
Downloaded: 7.html
Downloaded: 8.html
Downloaded: 9.html
Downloaded: 10.html
Downloaded: 11.html
Downloaded: 12.html
Downloaded: 13.html
Downloaded: 14.html
Downloaded: 15.html
Downloaded: 16.html
Downloaded: 17.html
Downloaded: 18.html
Downloaded: 19.html
Downloaded: 20.html
Downloaded: 21.html
Downloaded: 22.html
Downloaded: 23.html
Downloaded: 24.html
Downloaded: 25.html
Downloaded: 26.html
Downloaded: 27.html
Downloaded: 28.html
Downloaded: 29.html
Downloaded: 30.html
Downloaded: 31.html
Downloaded: 32.html
Downloaded: 33.html
Downloaded: 34.html
Downloaded: 35.html
Downloaded: 36.html
Downloaded: 37.html
Downloaded: 38.html
Downloaded: 39.html
Downloaded: 40.html
Downloaded: 41.html
Downloaded: 42.html
Downloaded: 43.html
Downloaded: 44.html
Downloaded: 45.html
Downloaded: 46.html
Downloaded: 47.html
Downloaded: 48.html
Downloaded: 49.html
Downloaded: 50.html
Downloade

In [ ]:
import os
import re
import csv
from bs4 import BeautifulSoup

# Load Indonesian stopwords from file
with open('stopwords.txt', 'r', encoding='utf-8') as file:
    stop_words = set(file.read().splitlines())

# Function to preprocess the content
def preprocess_content(content):
    # Convert to lowercase
    content = content.lower()

    # Remove special characters and numbers
    content = re.sub(r'[^a-z\s]', '', content)

    # Remove stopwords
    content = ' '.join([word for word in content.split() if word not in stop_words])

    return content

# Path to the folder containing the HTML files
html_files_dir = 'html_files/(kategori)'

# Function to extract content from a single HTML file
def extract_content_from_html(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')

        # Extract title, date, and author
        title = soup.find(class_='main-content-title')
        date = soup.find(class_='main-content-date')
        author = soup.find(class_='main-content-author')

        # Extract paragraphs in the 'main-content-detail'
        detail_container = soup.find(class_='main-content-detail')
        paragraphs = detail_container.find_all('p') if detail_container else []

        # Prepare the content
        title_text = title.get_text(strip=True) if title else 'N/A'
        date_text = date.get_text(strip=True) if date else 'N/A'
        author_text = author.get_text(strip=True) if author else 'N/A'

        # Use `get_text(' ')` to ensure spaces between text segments
        content_text = ' '.join(
            [preprocess_content(paragraph.get_text(' ', strip=True)) for paragraph in paragraphs]
        )

        return {
            'Title': title_text,
            'Date': date_text,
            'Author': author_text,
            'Content': content_text
        }

# List to store all the extracted data
all_data = []

# Process all HTML files in the folder
for filename in os.listdir(html_files_dir):
    if filename.endswith('.html'):
        file_path = os.path.join(html_files_dir, filename)
        content = extract_content_from_html(file_path)
        all_data.append(content)

# Path to the output CSV file
output_csv_path = 'data/militer/militer.csv'

# Create the output directory if it doesn't exist
os.makedirs('data', exist_ok=True)

# Write all data to a CSV file
with open(output_csv_path, 'w', encoding='utf-8', newline='') as csvfile:
    fieldnames = ['Title', 'Date', 'Author', 'Content']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for data in all_data:
        writer.writerow(data)

print(f"All extracted content saved to: {output_csv_path}")

All extracted content saved to: data/militer/militer.csv


In [4]:
import os
import re
import csv
from bs4 import BeautifulSoup
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Load Indonesian stopwords from file
with open('stopwords.txt', 'r', encoding='utf-8') as file:
    stop_words = set(file.read().splitlines())

# Initialize stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Function to preprocess the content
def preprocess_content(content):
    # Convert to lowercase
    content = content.lower()

    # Remove special characters and numbers
    content = re.sub(r'[^a-z\s]', '', content)

    # Remove stopwords
    content = ' '.join([word for word in content.split() if word not in stop_words])

    # Apply stemming
    content = ' '.join([stemmer.stem(word) for word in content.split()])

    return content

# Path to the folder containing the HTML files
html_files_dir = 'html_files/edukasi'

# Function to extract content from a single HTML file
def extract_content_from_html(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')

        # Extract title, date, and author
        title = soup.find(class_='main-content-title')
        date = soup.find(class_='main-content-date')
        author = soup.find(class_='main-content-author')

        # Extract paragraphs in the 'main-content-detail'
        detail_container = soup.find(class_='main-content-detail')
        paragraphs = detail_container.find_all('p') if detail_container else []

        # Prepare the content
        title_text = title.get_text(strip=True) if title else 'N/A'
        date_text = date.get_text(strip=True) if date else 'N/A'
        author_text = author.get_text(strip=True) if author else 'N/A'

        # Use `get_text(' ')` to ensure spaces between text segments
        content_text = ' '.join(
            [preprocess_content(paragraph.get_text(' ', strip=True)) for paragraph in paragraphs]
        )

        return {
            'Title': title_text,
            'Date': date_text,
            'Author': author_text,
            'Content': content_text
        }

# List to store all the extracted data
all_data = []

# Process all HTML files in the folder
for filename in os.listdir(html_files_dir):
    if filename.endswith('.html'):
        file_path = os.path.join(html_files_dir, filename)
        content = extract_content_from_html(file_path)
        all_data.append(content)

# Path to the output CSV file
output_csv_path = 'data2/edukasi2/edukasi2.csv'

# Create the output directory if it doesn't exist
os.makedirs('data2/edukasi2', exist_ok=True)

# Write all data to a CSV file
with open(output_csv_path, 'w', encoding='utf-8', newline='') as csvfile:
    fieldnames = ['Title', 'Date', 'Author', 'Content']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for data in all_data:
        writer.writerow(data)

print(f"All extracted content saved to: {output_csv_path}")

All extracted content saved to: data2/edukasi2/edukasi2.csv
